# hashed-ezkl

Here's an example leveraging EZKL whereby the inputs to the model, and the model params themselves, are hashed inside a circuit.

In this setup:
- the hashes are publicly known to the prover and verifier
- the hashes serve as "public inputs" (a.k.a instances) to the circuit

We leave the outputs of the model as public as well (known to the  verifier and prover). 


First we import the necessary dependencies and set up logging to be as informative as possible. 

In [2]:
from torch import nn
import ezkl
import os
import json
import logging

# uncomment for more descriptive logging 
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.DEBUG)


Now we define our model. It is a humble model with but a conv layer and a $ReLU$ non-linearity, but it is a model nonetheless

In [3]:
import torch
# Defines the model
# we got convs, we got relu, 
# What else could one want ????
from typing import *

Field = int

def hasher(l: Field, r: Field): 
    return l + r

class MerkleTree():
    def __init__(self, elements: List[Field], levels: int):
        self.nextId = 0
        self.levels = levels
        for i in range
    
    def insert(self, leaf):
        currentIndex = self.nextId
        currentLevelHash = leaf

        for (uint32 i = 0; i < levels; i) {
            if (currentIndex % 2 == 0) {
                left = currentLevelHash;
                right = zeros(i);
                filledSubtrees[i] = currentLevelHash;
            } else {
                left = filledSubtrees[i];
                right = currentLevelHash;
            }
            currentLevelHash = hashLeftRight(hasher, left, right);
            currentIndex /= 2;
        }

        nextIndex = _nextIndex + 1;
        return _nextIndex;
        
def create_merkle_tree(elements: List[Field]):
    for e in elements:


class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

    def verify_merkle_proof(self, leaf: Field, index: Field, merkle_path: List[Field]):
        current_hash = leaf
        for sibling in merkle_path:
            if index % 2 == 0:
                current_hash = self.hashLR(current_hash, sibling)
            else:
                current_hash = self.hashLR(sibling, current_hash)
            
            index = index // 2

        return current_hash - merkle_path[-1]

    def hashLR(l: Field, r: Field) -> Field:
        return hasher(l, r)

    def forward(self, x):
        leaf, index, *path = x # assume len(path) == height of the tree
        return self.verify_merkle_proof(leaf, index, path)


circuit = MyModel()

# this is where you'd train your model




We omit training for purposes of this demonstration. We've marked where training would happen in the cell above. 
Now we export the model to onnx and create a corresponding (randomly generated) input file.

You can replace the random `x` with real data if you so wish. 

In [4]:
x = 0.1*torch.rand(1,*[3, 8, 8], requires_grad=True)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      "network.onnx",            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open("input.json", 'w' ))



================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



This is where the magic happens. We define our `PyRunArgs` objects which contains the visibility parameters for out model. 
- `input_visibility` defines the visibility of the model inputs
- `param_visibility` defines the visibility of the model weights and constants and parameters 
- `output_visibility` defines the visibility of the model outputs

There are currently 4 visibility settings:
- `public`: known to both the verifier and prover (a subtle nuance is that this may not be the case for model parameters but until we have more rigorous theoretical results we don't want to make strong claims as to this). 
- `private`: known only to the prover
- `hashed`: the hash pre-image is known to the prover, the prover and verifier know the hash. The prover proves that the they know the pre-image to the hash. 
- `encrypted`: the non-encrypted element and the secret key used for decryption are known to the prover. The prover and the verifier know the encrypted element, the public key used to encrypt, and the hash of the decryption hey. The prover proves that they know the pre-image of the hashed decryption key and that this key can in fact decrypt the encrypted message.

Here we create the following setup:
- `input_visibility`: "hashed"
- `param_visibility`: "hashed"
- `output_visibility`: public

We encourage you to play around with other setups :) 

Shoutouts: 

- [summa-solvency](https://github.com/summa-dev/summa-solvency) for their help with the poseidon hashing chip. 
- [timeofey](https://github.com/timoftime) for providing inspiration in our developement of the el-gamal encryption circuit in Halo2. 

In [5]:
import ezkl

model_path = os.path.join('network.onnx')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
data_path = os.path.join('input.json')

run_args = ezkl.PyRunArgs()
run_args.input_visibility = "hashed"
run_args.param_visibility = "hashed"
run_args.output_visibility = "public"





Now we generate a settings file. This file basically instantiates a bunch of parameters that determine their circuit shape, size etc... Because of the way we represent nonlinearities in the circuit (using Halo2's [lookup tables](https://zcash.github.io/halo2/design/proving-system/lookup.html)), it is often best to _calibrate_ this settings file as some data can fall out of range of these lookups.

You can pass a dataset for calibration that will be representative of real inputs you might find if and when you deploy the prover. Here we create a dummy calibration dataset for demonstration purposes. 

In [6]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)
assert res == True

DEBUG tract_onnx.model 2023-07-22 15:27:30,323 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-22 15:27:30,325 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-22 15:27:30,325 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:30,327 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:30,327 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:30,327 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:30,327 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:30,328 change_axes.rs:76   Considering change Axis

In [7]:
# generate a bunch of dummy calibration data
cal_data = {
    "input_data": [(0.1*torch.rand(40, *[3, 8, 8])).flatten().tolist()],
}

cal_path = os.path.join('val_data.json')
# save as json file
with open(cal_path, "w") as f:
    json.dump(cal_data, f)

res = await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

DEBUG tract_onnx.model 2023-07-22 15:27:30,355 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-22 15:27:30,356 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-22 15:27:30,357 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:30,358 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:30,358 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:30,358 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:30,359 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:30,359 change_axes.rs:76   Considering change Axis

As we use Halo2 with KZG-commitments we need an SRS string from (preferably) a multi-party trusted setup ceremony. For an overview of the procedures for such a ceremony check out [this page](https://blog.ethereum.org/2023/01/16/announcing-kzg-ceremony). The `get_srs` command retrieves a correctly sized SRS given the calibrated settings file from [here](https://github.com/han0110/halo2-kzg-srs). 

These SRS were generated with [this](https://github.com/privacy-scaling-explorations/perpetualpowersoftau) ceremony. 

In [8]:
res = ezkl.get_srs(srs_path, settings_path)


DEBUG reqwest.connect 2023-07-22 15:27:39,830 connect.rs:429 starting new connection: https://trusted-setup-halo2kzg.s3.eu-central-1.amazonaws.com/
INFO ezkl.execute 2023-07-22 15:27:40,873 execute.rs:455 SRS downloaded


We now need to generate the (partial) circuit witness. These are the model outputs (and any hashes) that are generated when feeding the previously generated `input.json` through the circuit / model. 

In [9]:
!export RUST_BACKTRACE=1

witness_path = "witness.json"

res = ezkl.gen_witness(data_path, model_path, witness_path, settings_path = settings_path)

DEBUG tract_onnx.model 2023-07-22 15:27:41,000 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-22 15:27:41,000 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-22 15:27:41,001 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,001 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,001 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,002 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,002 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,002 change_axes.rs:76   Considering change Axis

As a sanity check you can "mock prove" (i.e check that all the constraints of the circuit match without generate a full proof). 

In [10]:


res = ezkl.mock(witness_path, model_path, settings_path)

DEBUG tract_onnx.model 2023-07-22 15:27:41,099 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-22 15:27:41,100 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-22 15:27:41,100 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,101 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,101 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,101 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,101 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,101 change_axes.rs:76   Considering change Axis

Here we setup verifying and proving keys for the circuit. As the name suggests the proving key is needed for ... proving and the verifying key is needed for ... verifying. 

In [11]:
# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK
res = ezkl.setup(
        model_path,
        vk_path,
        pk_path,
        srs_path,
        settings_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

DEBUG tract_onnx.model 2023-07-22 15:27:41,990 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-22 15:27:41,990 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-22 15:27:41,991 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,991 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,991 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,992 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,992 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:41,992 change_axes.rs:76   Considering change Axis

Now we generate a full proof. 

In [12]:
# GENERATE A PROOF

proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        model_path,
        pk_path,
        proof_path,
        srs_path,
        "evm",
        "single",
        settings_path,
    )

print(res)
assert os.path.isfile(proof_path)

DEBUG tract_onnx.model 2023-07-22 15:27:46,914 model.rs:247 ONNX operator set version: 10
DEBUG tract_hir.infer.analyser 2023-07-22 15:27:46,915 analyser.rs:151   Refined 3/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_hir.infer.analyser 2023-07-22 15:27:46,915 analyser.rs:151   Refined 4/0>: ..,? -> 1,1,1,1,F32
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:46,916 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:46,916 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:46,916 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:46,916 change_axes.rs:88     Change AxisChange { outlet: 1/0>, op: Rm(0) } blocked by locked interface 3/0>
DEBUG tract_core.optim.change_axes 2023-07-22 15:27:46,917 change_axes.rs:76   Considering change Axis

{'instances': [[[0, 0, 0, 0]], [[10605269020406003412, 11390436772184365076, 6474296778739487873, 3155332539224990236], [0, 0, 0, 0]]], 'proof': '1ffb70a036f6038973a2e34353138ed6e8032a96d5a9c25b3dda6fa40fefa6c916e9c5dc6c5d627e9d70ba8416a1b0cf0f3b07fafc90037fa9062fcd269a4cb6216692b841bf890c05c1f6ea9734ac0e3ae571b2e40223cc84df06a46b2c03c410064b0a2d94c6e9422730140fd3343aa6f87ef6c75cd00e6fcc5cbc54005e4b1dea17fee547941f6aea5d12f82126a997cc78db9a3adfb08debcacc0e8f642b18387fe947503964bb7f3fa2678dc244026b721112cf58677bc631cff8c90e0f2712e139836030df8c7ad6b93f8178b01aad0ed5d1aba0ea129aae317ffb1ee90b2097917754d9446a0a23f2c0b46a0f4f7f4a304a0d56a9d5426d013241451024148a8dfec94a63d928080a2341697df78d682524e0435d2cf165327af54d2b196c9a4351336c6c9ecc708a0485f4971f0d09e5e3c99d1f2c487f72f2c8b503153bada2293d0c09b3a6d1ae605b106d6e7b3f70bd71de298e2ec3c5bbfcee3407582cd4c78bfd3c7728fd26a80bd8e944f3c2e8a120faba49640c800cae28710bb42ab4660209c15c1927605134a70d0124f09289a980a95d9c164153036675114b90cc80d2788532284d

And verify it as a sanity check. 

In [13]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

assert res == True
print("verified")

INFO ezkl.pfsys.srs 2023-07-22 15:27:52,764 srs.rs:23 loading srs from "kzg.srs"
INFO ezkl.execute 2023-07-22 15:27:52,768 execute.rs:1498 downsizing params to 15 logrows
INFO ezkl.pfsys 2023-07-22 15:27:52,769 mod.rs:508 loading verification key from "test.vk"
INFO ezkl.graph.model 2023-07-22 15:27:52,769 model.rs:572 configuring model
DEBUG ezkl.circuit.ops.chip 2023-07-22 15:27:52,770 chip.rs:341 assigning lookup input
DEBUG ezkl.circuit.ops.chip 2023-07-22 15:27:52,770 chip.rs:345 assigning lookup output
INFO ezkl.execute 2023-07-22 15:27:52,806 execute.rs:1457 verify took 0.32
INFO ezkl.execute 2023-07-22 15:27:52,807 execute.rs:1462 verified: true


verified


We can now create an EVM / `.sol` verifier that can be deployed on chain to verify submitted proofs using a view function.

In [15]:

abi_path = 'test.abi'
sol_code_path = 'test.sol'

res = ezkl.create_evm_verifier(
        vk_path,
        srs_path,
        settings_path,
        sol_code_path,
        abi_path,
    )
assert res == True


INFO ezkl.pfsys.srs 2023-07-22 15:34:31,406 srs.rs:23 loading srs from "kzg.srs"
INFO ezkl.execute 2023-07-22 15:34:31,415 execute.rs:1498 downsizing params to 15 logrows
INFO ezkl.pfsys 2023-07-22 15:34:31,417 mod.rs:508 loading verification key from "test.vk"
INFO ezkl.graph.model 2023-07-22 15:34:31,419 model.rs:572 configuring model
DEBUG ezkl.circuit.ops.chip 2023-07-22 15:34:31,425 chip.rs:341 assigning lookup input
DEBUG ezkl.circuit.ops.chip 2023-07-22 15:34:31,426 chip.rs:345 assigning lookup output


## Verify on the evm

In [18]:
# Make sure anvil is running locally first
# run with $ anvil -p 3030
# we use the default anvil node here
import json

address_path = os.path.join("address.json")

res = ezkl.deploy_evm(
    address_path,
    sol_code_path,
    'http://127.0.0.1:3030'
)

assert res == True

with open(address_path, 'r') as file:
    addr = file.read().rstrip()

DEBUG reqwest.connect 2023-07-22 15:37:27,320 connect.rs:429 starting new connection: http://127.0.0.1:3030/


INFO ezkl.eth 2023-07-22 15:37:27,342 eth.rs:77 using chain 31337
DEBUG ezkl.eth 2023-07-22 15:37:27,792 eth.rs:527 runtime bytecode size: 18800
INFO ezkl.execute 2023-07-22 15:37:27,819 execute.rs:911 Contract deployed at: 0x5fbdb2315678afecb367f032d93f642f64180aa3


In [19]:
# make sure anvil is running locally
# $ anvil -p 3030

res = ezkl.verify_evm(
    proof_path,
    addr,
    "http://127.0.0.1:3030"
)
assert res == True

INFO ezkl.eth 2023-07-22 15:37:30,237 eth.rs:218 public_inputs: [
    0,
    19806343357691248164380188708129346483240156189866504844048169401257243522772,
    0,
]
INFO ezkl.eth 2023-07-22 15:37:30,238 eth.rs:219 proof: Bytes(0x1ffb70a036f6038973a2e34353138ed6e8032a96d5a9c25b3dda6fa40fefa6c916e9c5dc6c5d627e9d70ba8416a1b0cf0f3b07fafc90037fa9062fcd269a4cb6216692b841bf890c05c1f6ea9734ac0e3ae571b2e40223cc84df06a46b2c03c410064b0a2d94c6e9422730140fd3343aa6f87ef6c75cd00e6fcc5cbc54005e4b1dea17fee547941f6aea5d12f82126a997cc78db9a3adfb08debcacc0e8f642b18387fe947503964bb7f3fa2678dc244026b721112cf58677bc631cff8c90e0f2712e139836030df8c7ad6b93f8178b01aad0ed5d1aba0ea129aae317ffb1ee90b2097917754d9446a0a23f2c0b46a0f4f7f4a304a0d56a9d5426d013241451024148a8dfec94a63d928080a2341697df78d682524e0435d2cf165327af54d2b196c9a4351336c6c9ecc708a0485f4971f0d09e5e3c99d1f2c487f72f2c8b503153bada2293d0c09b3a6d1ae605b106d6e7b3f70bd71de298e2ec3c5bbfcee3407582cd4c78bfd3c7728fd26a80bd8e944f3c2e8a120faba49640c800cae28710bb